# Data Augmentation

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from tqdm import tqdm
import pandas as pd
from wordcloud_fa import WordCloudFa

from hazm import word_tokenize
from nltk.util import ngrams

In [2]:
tqdm.pandas()

In [3]:
base_path = "../data/"

tweets_path = base_path + 'remote_tweets.csv'

In [4]:
tweets_df = pd.read_csv(tweets_path, low_memory=False)
tweets_df.head()

,conversation_id,retweets_count,mentions,created_at,likes_count,in_reply_to_user_id,_id,source,user_id,in_reply_to_status_id,tweet,replies_count,quoted_status_id,hashtags,cashtags,urls
0,1375559420176908296,3,NaN,2021-03-26 21:24:57+00:00,163,NaN,1375559420176908296,Twitter for iPhone,812764546012246017,NaN,اینایی که رفتن سفر، مهمونی و... هیچ پروتکلی هم...,1,NaN,NaN,NaN,NaN
1,1383828354197979136,2,NaN,2021-04-18 17:02:44+00:00,37,NaN,1383828354197979136,Twitter for Android,985101690536898565,NaN,دارم فکر میکنم که چقدر خسته ام با اینکه امروز ...,3,NaN,NaN,NaN,NaN
2,1383840315031781385,0,NaN,2021-04-18 17:50:16+00:00,11,NaN,1383840315031781385,Twitter Web App,310871673,NaN,یک سال و نیمه از قبل آبان ۹۸ دورکار شدم تا الان,1,1.383657e+18,NaN,NaN,NaN
3,1383820625354002439,1,['15782990'],2021-04-18 17:31:42+00:00,9,15782990.0,1383835643239747593,Twitter Web App,1271095835028307969,1.383821e+18,@shahrzaad حتمن میرفتم به شهری که به اقوام یا ...,1,NaN,NaN,NaN,NaN
4,1383830770125443083,0,NaN,2021-04-18 17:12:20+00:00,8,NaN,1383830770125443083,Twitter Web App,897798038563033088,NaN,دورکاری یا کار از خونه خیلی برام فرسایشی شده د...,2,NaN,NaN,NaN,NaN


In [10]:
def get_uni_bi_tri_grams(text):
    tokens = word_tokenize(text)
    bigrams = list(ngrams(tokens, 2)) if len(tokens) >= 2 else []
    bigram_tokens = [' '.join(b) for b in bigrams]
    trigrams = list(ngrams(tokens, 3)) if len(tokens) >= 3 else []
    trigram_tokens = [' '.join(t) for t in trigrams]
    tokens = tokens + bigram_tokens + trigram_tokens
    return tokens

In [11]:
tweets_df['tokens'] = tweets_df['tweet'].progress_apply(lambda t: get_uni_bi_tri_grams(t))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 43441/43441 [00:02<00:00, 17610.78it/s]


In [19]:
def get_word_counts(data_df):
    word_counts = {}
    for index, row in tqdm(data_df.iterrows()):
        for token in row['tokens']:
            if token in word_counts:
                word_counts[token] += 1
            else:
                word_counts[token] = 1
    return word_counts

tweet_word_counts = get_word_counts(tweets_df)

43441it [00:03, 13635.42it/s]


In [23]:
sorted_words = dict(sorted(tweet_word_counts.items(), key=lambda item: item[1], reverse=True))
sorted_words

{'.': 54829,
 'و': 40166,
 'دورکاری': 29595,
 '،': 23979,
 'از': 22041,
 'که': 21832,
 'به': 21505,
 ':': 20816,
 'در': 19255,
 'کار': 18273,
 'رو': 11268,
 'هم': 11104,
 'با': 10908,
 '//t': 10585,
 ': //t': 10585,
 '//t .': 10585,
 ': //t .': 10585,
 'این': 10536,
 ')': 10322,
 'https': 9812,
 'https :': 9812,
 'https : //t': 9812,
 '. .': 8644,
 'برای': 8227,
 'من': 7487,
 'تو': 7214,
 '؟': 6997,
 'یه': 6443,
 '!': 5942,
 'دورکار': 5574,
 'تا': 5306,
 'کار در': 4852,
 ') )': 4798,
 '. . .': 4250,
 'یا': 4239,
 'منزل': 4062,
 'ما': 3988,
 'یک': 3950,
 'را': 3786,
 'شرکت': 3774,
 'ولی': 3737,
 'خیلی': 3728,
 'باید': 3700,
 'در منزل': 3674,
 'کرونا': 3568,
 '#دورکاری': 3414,
 'خونه': 3304,
 '(': 3227,
 'کار در منزل': 3207,
 'روز': 3095,
 'دیگه': 3086,
 'است': 2887,
 'بعد': 2872,
 'همه': 2867,
 ': )': 2838,
 'خانه': 2771,
 'نیست': 2762,
 'بود': 2710,
 'ها': 2701,
 'کنم': 2698,
 'شده': 2623,
 'انجام': 2621,
 'میشه': 2595,
 ') ) )': 2585,
 'داره': 2569,
 'های': 2568,
 'می': 2553,
 'باشه':